In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc

In [3]:
num_folds = 5

In [35]:
def non_discriminative(dataset, num_latent):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
        train_sc, test_sc = reshape_for_sc(train), reshape_for_sc(test)
        train_data = np.array([train_sc[:, :, i ] for i in range(1, train.shape[1])]).swapaxes(1, 2)
        c = SparseCoding()
        c.train(train_data, num_latent=num_latent)
        pred = c.disaggregate(test_sc[:, :, 0].swapaxes(0, 1)).swapaxes(0, 2).swapaxes(1, 2)
        pred = pred.reshape(pred.shape[0], pred.shape[1], -1, 24)
        out.append(pred)
    return np.concatenate(out)


In [43]:
dataset = 3
# tensor = np.load('../2015-5appliances.numpy.npy')
tensor = np.load("../2015-5appliances-subtract-true-agg.npy")
from sklearn.metrics import mean_absolute_error

err_non_disc ={}
gt = tensor[:, 1:, :, :]
for num_latent in range(1, 30):
    print(num_latent)
    pred = non_discriminative(dataset, num_latent)
    # Clamping prediction to aggregate
    pred = np.minimum(pred, tensor[:, 0:1, :,:])
    err_non_disc[num_latent] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [44]:
pd.DataFrame(err_non_disc).mean().argmin()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  if __name__ == '__main__':


26

In [45]:
pd.DataFrame(err_non_disc)[pd.DataFrame(err_non_disc).mean().argmin()]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  if __name__ == '__main__':


dr        120.500508
dw         76.699079
fridge    371.988901
hvac      422.649025
mw        168.048767
Name: 26, dtype: float64

In [16]:
pred[:,1]

array([[[ 473.76465099,  376.6603031 ,  327.48857859, ...,
          399.09705103,  312.49855986,  425.57684109],
        [ 319.4129581 ,  364.81687655,  498.61700971, ...,
          366.98777216,  306.66593295,  300.75577944],
        [ 300.75300844,  298.72606096,  395.19772955, ...,
          404.81637567,  479.22669411,  375.7917511 ],
        ...,
        [ 406.16603749,  385.86159297,  401.22660658, ...,
          409.05806972,  384.775062  ,  380.45823237],
        [ 451.46647957,  340.72268739,  395.73658218, ...,
          443.51079895,  249.89377148,  438.24458285],
        [ 436.66664875,  393.71207217,  456.28501835, ...,
          502.68590305,  434.32616885,  380.63380111]],

       [[ 402.04602405,  283.06120234,  311.0381636 , ...,
          300.95338692,  384.50578903,  382.55475809],
        [ 285.16758461,  303.6810313 ,  505.71641371, ...,
          262.64975268,  199.80390159,  196.67112103],
        [ 196.99354154,  267.03763551,  356.28899933, ...,
          354.